## Complexity Metrics of Cyber Network
##### Gabe Weaver, 3.15.19
For this experiment, we are going to consider the structural complexity of a cyber network.  A snippet of the network is shown below.

![](images/city_of_arcoTest.png)

In [10]:
from edu.illinois.iti.dao.CommunicationsNetworkDAO import IMNCommunicationsNetworkDAO
import json
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go


# 1.  Read in the network
networkFilePath = "../data/testing/city_of_arcoTest.imn"
cnDAO = IMNCommunicationsNetworkDAO.create(networkFilePath)
gCyber = cnDAO.getNetwork(networkFilePath)

### 2.  Compute Complexity Metrics
In this step, we compute complexity metrics on the graph.  When a complexity metric is applied to a graph, the implementation will decorate that graph with an attribute whose value is the output of that metric.

#### Degree
The degree of a node is a simple measure to understand its importance.

In [13]:
def computeNodeDegrees(G):
    """
    Decorate nodes in the graph with an attribute 
      name 'nx:in_degree', 'nx:out_degree'.  The value 
      of this attribute is an integer that is the 
      degree of the node.
    """
    for nIdx in G.nodes():
        node = G.nodes[nIdx]
        node['nx:in_degree'] = G.in_degree(nIdx)
        node['nx:out_degree'] = G.out_degree(nIdx)
        node['gc:degree'] = G.in_degree(nIdx) + G.out_degree(nIdx)
    return G    

def computeMetrics(G, metrics):
    for metric in metrics:
        if "gc:degree" == metric:
            computeNodeDegrees(G)
        else:
            raise NotImplementedError() 
    return

### 4.  See How Complexity Metrics Affect Performance Measures


In [38]:
# Compute complexity metrics of interest
metrics = ["gc:degree"]
computeMetrics(gCyber, metrics)

# Sort nodes by a metric value 
nodes = gCyber.nodes(data=True)
sNodes = sorted(nodes, key=lambda x: x[1]['gc:degree'], reverse=True)

df = pd.DataFrame()
for node in sNodes:
    row = pd.Series( node[1] )
    df = df.append(row, ignore_index=True)
df.head()

,gc:degree,imn:hostname,imn:interface,name,nx:in_degree,nx:out_degree
0,7.0,RJ45PP-02-1,e0,n11,5.0,2.0
1,6.0,ENS-02-3,e0,n3,0.0,6.0
2,6.0,ENS-5-1,e0,n90,0.0,6.0
3,5.0,ENS-2-1,e0,n16,4.0,1.0
4,5.0,ENS-15-1,e0,n19,3.0,2.0


So with respect to the degree metric, we should be looking at the effect of disrupting the assets in the table above on some Key Performance Indicators within the water treatment plant's cyber network.  

From looking at the diagram above:
* RJ45PP-02-01 connects the Plant 1 Operation's Room to the Plant 1 Computer room
* ENS-02-03 is in the Plant 1 Computer Room
* ENS-5-1 is in the Plant 2 Operations Room

**Ask Tim, etc:  What KPI should we be using to capture normal behavior?**


### 5.  Visualize Critical Nodes
Next in the pipeline will be to visualize critical nodes relative to some metric value.

In [3]:
#  Based on https://plot.ly/python/network-graphs

def plotGraph(G, metricKey, nodeLabelKey):
    pos = nx.get_node_attributes(G,'pos')
    
    #-- Get Node positions
    dmin=1
    ncenter=0
    for n in pos:
        x,y=pos[n]
        d=(x-0.5)**2+(y-0.5)**2
        if d<dmin:
            ncenter=n
            dmin=d

    p=nx.single_source_shortest_path_length(G,ncenter)

    #-- Create Edges
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])

    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))

    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        
    # Color Node Points By Metric Value
    for nIdx in G.nodes():
        node = G.nodes[nIdx]
        metricValue = node[metricKey]
        
        labelValue = ""
        if nodeLabelKey in node:
            labelValue = node[nodeLabelKey]
        else:
            print(node)

        node_trace['marker']['color']+=tuple([metricValue])
        node_info = labelValue
        node_trace['text']+=tuple([node_info])

    # Create Network Graph
    fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))
    return fig


In [4]:
plotly.tools.set_credentials_file(username='gweaver', api_key='CWI1yxjAs6jKd2ICgW7y')
plotly.tools.set_config_file(world_readable=True)

# Set node positions according to a layout
nodePositions = nx.spring_layout(gCyber)
for nIdx, posValue in nodePositions.items():
    gCyber.nodes[nIdx]['pos'] = posValue

computeNodeDegrees(gCyber)
fig = plotGraph(gCyber, metricKey="gc:degree",nodeLabelKey="imn:hostname")
py.iplot(fig, filename='networkx')

/home/tater/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

